In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [2]:
fileName = '../../1000_population.tsv'


lines = []
categories = []


with open(fileName, 'r') as f:
    text = f.read()

lines = text.split('\n')
for l in lines:
    
    categories.append(l.split('\t')[-1])
allcategories=sorted(list(set(categories)))


In [ ]:
def getAveragePer1000(filepath,n,filename):
    csv_df= pd.read_csv(filepath, low_memory = False)
    # csv_df = pd.DataFrame(csv_data)
    for col in allcategories:
        averages = [np.mean(csv_df.loc[i:i+n-1, col]) for i in range(0, csv_df.shape[0], n)]
        np.savetxt('/data2/wangxuedong/mhc_test_data/similar_region_race_average100/'+col+"_"+filename+'.csv', averages, delimiter = ',')


In [ ]:
#run 
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race/"):
    for file in files:
        filename=file.split(".")[0]
        getAveragePer1000(filepath=curDir+file,n=1000,filename=file.split(".")[0])

In [ ]:
arrforDistribution_different_race=dict()
for category in allcategories:
    arrforDistribution_different_race[category]=[]

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race_average100/"):
    for file in files:
        for category in allcategories:
            if file.startswith(category):
                csv_data=np.loadtxt(open(curDir+file,"rb"),delimiter=",",skiprows=0)
                arrforDistribution_different_race[category]+=csv_data.tolist()

In [ ]:
mean_dict_different_race=dict()
std_dict_different_race=dict()
for category in allcategories:
    mean_dict_different_race[category]=0
    std_dict_different_race[category]=0

for category in allcategories:
    mean_dict_different_race[category]=np.mean(arrforDistribution_different_race[category])
    std_dict_different_race[category]=np.std(arrforDistribution_different_race[category])

In [5]:
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):

    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../../similar_region_race_95_comparewith/"+filename+".csv", arr, delimiter = ',')
    return filename

In [6]:
for curDir, dirs, files in os.walk(top="../../similar_region_race_average100/"):
    for file in files:   
        for category in allcategories:
            if file.startswith(category):
                filename=file[:file.index(".")]
                filepath=os.path.join(curDir,file)
                #print(filepath)
                compareWith0_01Pvalue(mean_arr=mean_dict_different_race[category],std_arr=std_dict_different_race[category],filename=filename,filepath=filepath)

In [7]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../../similar_region_race_95_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [8]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        if arr[i]==1:
            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1                    
                else:                   
                    break
            res[i]=count
            i+=count
        else:
            i+=1
            continue
    return res
            


In [9]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [10]:
for curDir, dirs, files in os.walk(top="../../similar_region_race_95_comparewith/"):
    for file in files:

        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [11]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    dic = {k: v for k, v in sorted(dic.items())}

    return dic

In [12]:

race_dic_except_mhc=dict()
for category in allcategories:
    race_dic_except_mhc[category]=dict()

for curDir, dirs, files in os.walk(top="../../similar_region_race_95_start_length/"):
    for category in allcategories:
        mydic=dict()
        for file in files:        
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    continue          
                else:                  
                    filepath=os.path.join(curDir,file)
                    mydic=countNumofLength(mydic,filepath=filepath)
        race_dic_except_mhc[category]=mydic

In [13]:
race_dic_except_mhc

{'ACB': {1: 305,
  2: 118,
  3: 38,
  4: 22,
  5: 15,
  6: 14,
  7: 8,
  8: 5,
  9: 4,
  10: 2,
  12: 3,
  13: 2,
  14: 1,
  15: 2,
  17: 1,
  19: 1,
  20: 1,
  21: 2,
  30: 1,
  56: 1,
  63: 1,
  79: 1,
  82: 1,
  100: 1},
 'ASW': {1: 321,
  2: 107,
  3: 42,
  4: 20,
  5: 19,
  6: 11,
  7: 6,
  8: 5,
  9: 1,
  10: 1,
  11: 1,
  12: 3,
  13: 2,
  15: 2,
  16: 1,
  17: 3,
  20: 1,
  21: 1,
  22: 1,
  30: 1,
  44: 1,
  56: 1,
  62: 1,
  79: 1,
  82: 1,
  99: 1},
 'BEB': {1: 353,
  2: 111,
  3: 51,
  4: 28,
  5: 18,
  6: 11,
  7: 4,
  8: 1,
  9: 3,
  10: 1,
  11: 2,
  13: 1,
  14: 1,
  16: 2,
  17: 1,
  20: 1,
  21: 1,
  27: 1,
  31: 1,
  35: 1,
  42: 1,
  45: 2,
  56: 1,
  65: 1,
  100: 1},
 'CDX': {1: 329,
  2: 94,
  3: 38,
  4: 27,
  5: 15,
  6: 6,
  7: 5,
  8: 4,
  9: 2,
  10: 4,
  11: 1,
  12: 1,
  13: 2,
  14: 2,
  15: 1,
  16: 1,
  20: 1},
 'CEU': {1: 366,
  2: 111,
  3: 55,
  4: 37,
  5: 19,
  6: 9,
  7: 10,
  8: 5,
  9: 2,
  10: 2,
  11: 2,
  12: 1,
  13: 1,
  18: 2,
  19: 2,
  2

In [14]:
mhc_final_dict=dict()
for curDir, dirs, files in os.walk(top="../../similar_region_race_95_start_length/"):
    for category in allcategories:
        mhcdic=dict()
        for file in files:   
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    filepath=os.path.join(curDir,file)
                    mhcdic=countNumofLength(mhcdic,filepath=filepath)
        mhc_final_dict[category]=mhcdic
                  

In [15]:
mhc_final_dict

{'ACB': {},
 'ASW': {},
 'BEB': {},
 'CDX': {},
 'CEU': {},
 'CHB': {},
 'CHS': {},
 'CLM': {},
 'ESN': {},
 'FIN': {},
 'GBR': {},
 'GIH': {},
 'GWD': {},
 'IBS': {},
 'ITU': {},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {},
 'MXL': {},
 'PEL': {},
 'PJL': {},
 'PUR': {},
 'STU': {},
 'TSI': {},
 'YRI': {}}

In [16]:
arr1=[]
for _,v in race_dic_except_mhc.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)
for _,v in mhc_final_dict.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)

In [17]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 2.647185419328805
dev 6.277844744534425


42.58269589190771

In [18]:
def WithThreshold(threshold,dic):#dic={asian:{1:2,2:3,3:1},europe:{1:2,2:3,3:1}}
    larger_than_thres=0
    resultdic=dict()
    for category in allcategories:
        for i in dic[category].keys():
            if i>=threshold:
                larger_than_thres+=dic[category][i]
            resultdic[category]=larger_than_thres
    return resultdic


In [19]:

def getAllPositions(filepath):
    pos=[]
    #vcf_in=vcf(filepath)
    vcf_file = pysam.VariantFile(filepath)
    for record in vcf_file:

        pos.append(record.pos)
    pos=sorted(list(set(pos)))
    return pos

In [20]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [21]:

def GetPosition(filepath,threshold,res):

    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]

    filename_components=(filepath.split("/")[-1]).split("_start_lenth.csv")[0].split("_")
    #print(filename_components)
    findfoldername="split_"+filename_components[2]
    findfoldername_filename=filename_components[3]
    filepath="/data2/wangxuedong/mhc_test_data/"+findfoldername+"/"+findfoldername_filename+".vcf.gz"
    print(filepath)
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    # category=filepath.split("/")[2].split("_")
    if midarr!=[]:
        res[findfoldername+"/"+findfoldername_filename]=midarr
    return res

In [22]:

def GetPositionformhc(filepath,threshold,res):

    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
        
    filepath="/data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf"

    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    # category=filepath.split("/")[2].split("_")
    if midarr!=[]:
        res["mhc"]=midarr
    return res

In [23]:
all_race_positons=dict()
for category in allcategories:
    all_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                continue
            else:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    res=GetPosition(filepath=filepath,threshold=43,res=res)
                    all_race_positons[category]=res


/data2/wangxuedong/mhc_test_data/split_chr1/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaa.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaq.vcf.gz
/data2/wan

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr1/xbb.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr1/xay.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xba.vcf.gz
filepath /data2/wangxuedong/mhc_test_data/split_chr1/xba.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr1/xba.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr1/xat.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaw.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbc.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbe.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaz.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xax.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbg.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbh.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbd.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbi.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbk.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xby.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbl.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbm.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xbs.vcf.gz
/data2/wan

...

/data2/wangxuedong/mhc_test_data/split_chr12/xbc.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xav.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xaf

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr2/xav.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr2/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xau.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr19/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xax.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xat.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr18/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xaa.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr18/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr2/xaf.vcf.gz
/data2/

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr3/xau.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr3/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbi.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbo.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbd.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaa.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xas.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr21/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr21/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr22/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr22/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr5/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xbc.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr4/xbj.vcf.gz
/data2

In [24]:
all_race_positons

{'ACB': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16000187, 18181463]],
  'split_chr2/xav': [[91720904, 94530414]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'ASW': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr13/xaa': [[16040220, 18181463]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr2/xav': [[91720904, 94530414]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'BEB': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16165134, 17417031]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'CDX': {},
 'CEU': {'split_chr1/xbb': [[123992935, 124920289]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16446009, 18181463]],
  'split_chr2/xav': [[92076199, 94195617]]},
 'CHB': {},
 'CHS': {'split_chr1/xba': [[123102017, 12

In [25]:
mhc_race_positons=dict()
for category in allcategories:
    mhc_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    print(filepath)
                    res=GetPositionformhc(filepath=filepath,threshold=43,res=res)
                    mhc_race_positons[category]=res
                else:
                    continue
            else:
                continue


/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/ACB_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/ASW_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/BEB_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/CDX_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/CEU_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/CHB_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/CHS_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/CLM_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/ESN_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/FIN_mhc_start_lenth.csv
/data2/wangxuedong/mhc_test_data/similar_region_race_95_start_length/G

In [26]:
mhc_race_positons

{'ACB': {},
 'ASW': {},
 'BEB': {},
 'CDX': {},
 'CEU': {},
 'CHB': {},
 'CHS': {},
 'CLM': {},
 'ESN': {},
 'FIN': {},
 'GBR': {},
 'GIH': {},
 'GWD': {},
 'IBS': {},
 'ITU': {},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {},
 'MXL': {},
 'PEL': {},
 'PJL': {},
 'PUR': {},
 'STU': {},
 'TSI': {},
 'YRI': {}}